<center>
NNTI Assignment 1

November 2, 2023

Name: Aleksey Morshnev
Student ID: 7042691
Email: almo00008@stud.uni-saarland.de
</center>

In [3]:
# List of packages you want to check/install
packages_to_install = ["numpy", "pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia", "pandas"]

In [4]:
%%time
import importlib

for package_name in packages_to_install:
    try:
        importlib.import_module(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        print(f"{package_name} is not installed. Installing it now...")
        !conda install -y {package_name}

numpy is already installed.
pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia is not installed. Installing it now...
Solving environment: done

# All requested packages already installed.

pandas is already installed.
CPU times: user 253 ms, sys: 72.6 ms, total: 326 ms
Wall time: 7.17 s


# Exercise 1

In [5]:
import numpy as np

In [6]:
x = np.random.rand(4, 16)
x

array([[0.04550158, 0.51511683, 0.86741125, 0.67078425, 0.29484577,
        0.41868973, 0.27516267, 0.84931748, 0.34744847, 0.55014228,
        0.89548734, 0.36057956, 0.93743805, 0.97767309, 0.269396  ,
        0.37150152],
       [0.00276488, 0.40221373, 0.61241231, 0.10468235, 0.69750856,
        0.28377318, 0.36101104, 0.86131137, 0.19978287, 0.96424361,
        0.44014827, 0.93990482, 0.05921269, 0.9719772 , 0.83305531,
        0.71964283],
       [0.74948089, 0.0383741 , 0.17474652, 0.670965  , 0.25031129,
        0.07608257, 0.82171381, 0.76029832, 0.57492581, 0.38441729,
        0.57097354, 0.82736696, 0.77216461, 0.55076766, 0.44004492,
        0.51178304],
       [0.28889222, 0.93363312, 0.62399447, 0.7720374 , 0.75274288,
        0.92488178, 0.70041117, 0.79104486, 0.83353621, 0.52855028,
        0.69869742, 0.47746754, 0.56647448, 0.3445859 , 0.81903138,
        0.25064436]])

In [7]:
w1, w2, w3 = [np.random.rand(*shape) for shape in [(16, 16), (32, 16), (2, 32)]]
w1, w2, w3

(array([[0.31471193, 0.6119991 , 0.72164407, 0.6772504 , 0.22187146,
         0.30497646, 0.8402459 , 0.55882955, 0.4944321 , 0.51487914,
         0.23756689, 0.84726512, 0.72477769, 0.35066281, 0.42628792,
         0.04227861],
        [0.4602319 , 0.98973719, 0.39527647, 0.18656558, 0.5941744 ,
         0.48163301, 0.42762415, 0.25963876, 0.06872325, 0.01322862,
         0.17898043, 0.14029477, 0.45634455, 0.19464341, 0.82011009,
         0.06034781],
        [0.41493001, 0.64987542, 0.95079456, 0.00235624, 0.22643123,
         0.40857704, 0.39556513, 0.646363  , 0.42720444, 0.06053353,
         0.75881594, 0.0975174 , 0.00318101, 0.50265911, 0.1252304 ,
         0.25155297],
        [0.43871123, 0.07582509, 0.30265403, 0.28963296, 0.15684644,
         0.40684099, 0.95291481, 0.08837379, 0.88627061, 0.43716978,
         0.77137709, 0.9605225 , 0.50234068, 0.68724342, 0.93626472,
         0.98900461],
        [0.50071378, 0.17293671, 0.93929225, 0.90323189, 0.85986226,
         0.1621

In [8]:
result = ((x @ w1.T) @ w2.T) @ w3.T # Result of forward pass
result

array([[587.19426929, 465.81360283],
       [580.75958977, 462.36828476],
       [562.52581992, 447.00342931],
       [679.34113503, 540.11173193]])

# (Bonus)

In [9]:
fourByFour = np.random.rand(4, 4)

In [10]:
eigenvalues, eigenvectors = np.linalg.eig(fourByFour)
eigenvalues, eigenvectors

(array([ 1.94227321+0.j        , -0.3636813 +0.15271295j,
        -0.3636813 -0.15271295j,  0.35727011+0.j        ]),
 array([[ 0.37200088+0.j        ,  0.21691123-0.34085894j,
          0.21691123+0.34085894j, -0.03740812+0.j        ],
        [ 0.79077668+0.j        , -0.51136226-0.09386243j,
         -0.51136226+0.09386243j, -0.83404175+0.j        ],
        [ 0.36504887+0.j        , -0.16357208+0.29463928j,
         -0.16357208-0.29463928j,  0.52596946+0.j        ],
        [ 0.32097804+0.j        ,  0.67297478+0.j        ,
          0.67297478-0.j        , -0.16226867+0.j        ]]))

# Exercise 2

## Part1

In [11]:
import json
import torch
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda")

In [12]:
class CustomDataset(Dataset):
    def __init__(self, json_file, train=True, transform=None):
        with open(json_file, 'r') as f:
            data = json.load(f)

        # Split data into training and testing datasets
        split_ratio = 0.8
        split_index = int(len(data) * split_ratio)
        if train:
            self.data = data[:split_index]
        else:
            self.data = data[split_index:]

        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample

In [13]:
def custom_collate_fn(batch):
    max_seq_len = max(len(sample['tokens']) for sample in batch)

    padded_inputs = []
    labels = []
    for sample in batch:
        input_sequence = sample['tokens']
        label = sample['label']

        padding = [0] * (max_seq_len - len(input_sequence))
        padded_input_sequence = input_sequence + padding

        padded_inputs.append(padded_input_sequence)
        labels.append(label)

    inputs = torch.tensor(padded_inputs)
    labels = torch.tensor(labels)

    return inputs, labels


In [14]:
class CustomDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None):
        super(CustomDataLoader, self).__init__(
            dataset, batch_size, shuffle, sampler, batch_sampler, num_workers, collate_fn
        )

    def __iter__(self):
        for batch in super(CustomDataLoader, self).__iter__():
            yield batch

In [15]:
dataset_path = './data/Dataset1/data.json'

train_dataset = CustomDataset(dataset_path, train=True)
train_loader = CustomDataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

for batch in train_loader:
    inputs, outputs = batch
    
test_dataset = CustomDataset(dataset_path, train=False)
test_loader = CustomDataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

for batch in test_loader:
    inputs, outputs = batch

## Part 2

In [16]:
import pandas as pd

In [31]:
class SickDataset(Dataset):
    def __init__(self, csv_file, train=True):
        self.data = pd.read_csv(csv_file)
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Select numeric columns and convert to tensor
        numeric_inputs = torch.tensor(self.data.select_dtypes(include=['float', 'int']).iloc[idx, :].values, dtype=torch.float32).to(device)
    
        # Select categorical columns, apply one-hot encoding, and convert to tensor
        categorical_inputs = torch.tensor(pd.get_dummies(self.data.select_dtypes(include=['object']).iloc[idx, :].values).values, dtype=torch.float32).to(device)
    
        # Reshape tensors to handle the case where they are one-dimensional
        numeric_inputs = numeric_inputs.view(-1, numeric_inputs.shape[0])
        categorical_inputs = categorical_inputs.view(-1, categorical_inputs.shape[0])
    
        # Concatenate numeric and categorical tensors
        inputs = torch.cat((numeric_inputs, categorical_inputs), dim=1).to(device)
    
        # Apply one-hot encoding to the entire target column
        target_column = pd.get_dummies(self.data.iloc[:, -1].values)
    
        # Extract the one-hot encoded vector for the current index
        target = torch.tensor(target_column.iloc[idx, :].values, dtype=torch.float32).to(device)
    
        return inputs, target

class SickDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, num_workers=0):
        super(SickDataLoader, self).__init__(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    def __iter__(self):
        for batch in super(SickDataLoader, self).__iter__():
            inputs, outputs = zip(*batch)
            yield torch.stack(inputs).to(device), torch.stack(outputs).to(device)

sick_dataset_path = './data/Dataset2/data.csv'

sick_dataset = SickDataset(sick_dataset_path, train=True)

custom_dataloader = CustomDataLoader(sick_dataset, batch_size=32, shuffle=True)

for batch in custom_dataloader:
    inputs, outputs = batch